# Fake address gen

In [2]:
import pandas as pd
import json
import re
import random
import string as string_lib

## Word list for fake address generation

Taken from <https://github.com/duyet/vietnamese-wordlist>, using the 11K words file.

In [3]:
with open("./Data/Viet11K.txt", 'r', encoding="utf-8") as file:
    vn_words = file.read().splitlines()
vn_words

['a',
 'a dua',
 'a hoàn',
 'a phiến',
 'a tòng',
 'à',
 'ả',
 'ả đào',
 'ả giang hồ',
 'á',
 'á-căn-đình',
 'á khẩu',
 'á khôi',
 'á kim',
 'á rập',
 'ạ',
 'ác',
 'ác cảm',
 'ác chiến',
 'ác mộng',
 'ác nghiệt',
 'ác phụ',
 'ác tà',
 'ác tâm',
 'ác thú',
 'ách',
 'ạch',
 'ai',
 'ai ai',
 'ai điếu',
 'ai oán',
 'ải',
 'ải quan',
 'ái',
 'ái ân',
 'ái hữu',
 'ái khanh',
 'ái lực',
 'ái mộ',
 'ái nam ái nữ',
 'ái ngại',
 'ái nhĩ lan',
 'ái phi',
 'ái quốc',
 'ái tình',
 'am',
 'am hiểu',
 'am pe',
 'ảm đạm',
 'ám',
 'ám ảnh',
 'ám chỉ',
 'ám hại',
 'ám hiệu',
 'ám muội',
 'ám sát',
 'ám tả',
 'ám thị',
 'an',
 'an ba ni',
 'an bài',
 'an cư',
 'an dưỡng',
 'an nghỉ',
 'an ninh',
 'an phận',
 'an táng',
 'an tâm',
 'an toàn',
 'an ủi',
 'an vị',
 'án',
 'án mạng',
 'án ngữ',
 'án phí',
 'án sát',
 'án thư',
 'án tử hình',
 'ang',
 'ang áng',
 'áng',
 'anh',
 'anh ánh',
 'anh dũng',
 'anh đào',
 'anh em',
 'anh hùng',
 'anh linh',
 'anh tài',
 'anh thư',
 'anh tuấn',
 'ảnh',
 'ảnh ảo',
 'ả

## Import adminstrative region data

In [4]:
with open("./Data/region_tree.json", 'r') as file:
    region_tree = json.load(file)
    
with open("./Data/reverse_index_dist_to_prov.json", 'r') as file:
    rev_index_dist_to_prov = json.load(file)
    
with open("./Data/reverse_index_ward_to_prov.json", 'r') as file:
    rev_index_ward_to_prov = json.load(file)
    
with open("./Data/reverse_index_ward_to_dist.json", 'r') as file:
    rev_index_ward_to_dist = json.load(file)

## String deterioration function

In [5]:
def expand_space_within_string(string: str, limit: (int, int), /, 
                               include_newline=False, newline_prob=0.05, 
                               include_tab=False, tab_prob=0.05) -> str:
    tokens = string.split()
    newline_prob = (newline_prob if include_newline else 0)
    tab_prob = (tab_prob if include_tab else 0)
    
    space_prob = 1 - newline_prob - tab_prob
    prob_weight = [space_prob, newline_prob, tab_prob]
    whitespace = [' ', '\n', '\t']
    result = str()
    
    for i, token in enumerate(tokens):
        result += token
        if i == len(tokens):
            pass
        num_space = random.randrange(*limit)
        spaces = ''.join(random.choices(whitespace, prob_weight, k=num_space))
        result += spaces
    
    return result

In [6]:
test = 'this is a test string'
expand = expand_space_within_string(
    test, (2, 8), 
    include_newline=True, newline_prob=0.1, 
    include_tab=True, tab_prob=0.05)
print(expand)

this     is 

   a      test 	string   	


In [7]:
# by default, string will deteriorate by having their chars substituted
def substitute_string(string:str) -> str:
    position = random.randrange(0, len(string))
    replace_with = string_lib.ascii_letters + string_lib.digits + string_lib.punctuation
    
    return string[:position] + random.choice(replace_with) + string[position+1:]
    
    
def insert_string(string:str) -> str:
    position = random.randrange(0, len(string))
    insert_with = string_lib.ascii_letters + string_lib.digits + string_lib.punctuation
    
    return string[:position] + random.choice(insert_with) + string[position:]


def delete_string(string:str) -> str:
    position = random.randrange(0, len(string))
    
    return string[:position] + string[position+1:]


def transpose_string(string:str) -> str:
    if len(string) < 2:
        return string
    
    position = random.randrange(0, len(string) - 1)
    
    return string[:position] + string[position+1] + string[position] + string[position+2:]
 
    
# substitution always on by default    
def deteriorate_string(string:str, edit_distance=1, /,
                      allow_insertion=True, allow_deletion=True, allow_transposition=True, 
                      insert_prob=0.1, delete_prob=0.1, trans_prob=0.05) -> str:
    
    insert_prob = (insert_prob if allow_insertion else 0)
    delete_prob = (delete_prob if allow_deletion else 0)
    trans_prob = (trans_prob if allow_transposition else 0)
    
    sub_prob = 1 - insert_prob - delete_prob - trans_prob
    action_weights = [sub_prob, insert_prob, delete_prob, trans_prob]
    actions = [substitute_string, insert_string, delete_string, transpose_string]
    
    for _ in range(edit_distance):
        action = random.choices(actions, action_weights, k=1)[0]
        string = action(string)
        
    return string

In [8]:
print(substitute_string('test'))
print(insert_string('test'))
print(delete_string('test'))
print(transpose_string('test'))

tBst
tesKt
tet
tset


In [9]:
deteriorate_string('this is a test string, it should be destroy incrementally', 6,
                  allow_insertion=True, allow_deletion=True, allow_transposition=True)

'this i[ a 6e:t string ,it should be deztroy incrementalIy'

## Abbreviation

In [10]:
def get_abbre_with_word_first_letter(string: str, /, with_sep=False, sep='.') -> str:
    
    words = string.strip().split()
    if len(words) < 2:
        return string
    
    for i, word in enumerate(words):
        words[i] = word.capitalize()
    
    sep = sep if with_sep else ''
    result = ''
    
    for word in words:
        if not word.isalnum():
            result += word[0]
        else:
            result += word[0] + (sep if with_sep else '')
    
    return result

In [11]:
print(get_abbre_with_word_first_letter('Hồ Chí Minh - Đà Nẵng'))
print(get_abbre_with_word_first_letter('Hồ Chí Minh - Đà Nẵng', with_sep=True))

HCM-ĐN
H.C.M.-Đ.N.


In [4]:
def get_abbre_with_word_first_letter_incomplete_last_word(string: str, /, 
                                                          with_sep=False, sep='.', *,
                                                          with_space_for_last_word=False) -> str:
    words = string.strip().split()
    if len(words) < 2:
        return string
    
    for i, word in enumerate(words):
        words[i] = word.capitalize()
    
    sep = sep if with_sep else ''
    result = ''
    
    for word in words[:-1]:
        if not word.isalnum():
            result += word[0]
        else:
            result += word[0] + (sep if with_sep else '')
    
    result += (' ' if with_space_for_last_word else '') + words[-1]
    
    return result

In [9]:
print(get_abbre_with_word_first_letter_incomplete_last_word('Hồ Chí Minh - Đà Nẵng'))
print(get_abbre_with_word_first_letter_incomplete_last_word('Hồ Chí Minh - Đà Nẵng', 
                                                            with_sep=True, sep='.', with_space_for_last_word=True))

HCM-ĐNẵng
H.C.M.-Đ. Nẵng


In [14]:
def get_abbre(string: str, /, full_abbr_prop=0.3, incomplete_abbr_prop=0.7, 
              with_sep_prob=0.6, with_space_for_last_word_prop=0.3) -> str:
    
    abbre_actions = [get_abbre_with_word_first_letter, get_abbre_with_word_first_letter_incomplete_last_word]
    abbre_actions_weight = [full_abbr_prop, incomplete_abbr_prop]
    action = random.choices(abbre_actions, abbre_actions_weight, k=1)[0]
    
    with_sep = random.random() < with_sep_prob
    with_space_for_last_word = random.random() < with_space_for_last_word_prop
    
    if action == get_abbre_with_word_first_letter_incomplete_last_word:
        result = action(string, with_sep=with_sep, with_space_for_last_word=with_space_for_last_word)
    else:
        result = action(string, with_sep=with_sep)
        
    #print(string, '-->', result)
    return result

In [15]:
print(get_abbre('Hồ Chí Minh - Đà Nẵng'))
print(get_abbre('Hồ Chí Minh - Đà Nẵng'))

H.C.M.-Đ.N.
H.C.M.-Đ.N.


## Fake address generation

In [16]:
def get_prov_dist_ward_level_region(region_tr) -> {'province': {}, 'district': {}, 'ward': {}}:
    random_prov_info = random.choice(list(region_tr.values()))
    random_dist_info = random.choice(list(random_prov_info['districts'].values()))
    random_ward_info = random.choice(list(random_dist_info['wards'].values()))
    
    return {
        'province': random_prov_info, 
        'district': random_dist_info, 
        'ward': random_ward_info
    }

def get_prov_dist_ward_level_region_name(selection: {
        'province': {}, 
        'district': {}, 
        'ward': {}
    }) -> {'province': str, 'district': str, 'ward': str}:
    
    return {
        'province': selection['province']['name'], 
        'district': selection['district']['name'], 
        'ward': selection['ward']['name']
    }

In [17]:
get_prov_dist_ward_level_region(region_tree)

{'province': {'prefix': 'Tỉnh',
  'name': 'Hoà Bình',
  'districts': {'Hòa Bình': {'prefix': 'Thành phố',
    'name': 'Hòa Bình',
    'wards': {'Thái Bình': {'prefix': 'Phường', 'name': 'Thái Bình'},
     'Tân Hòa': {'prefix': 'Phường', 'name': 'Tân Hòa'},
     'Thịnh Lang': {'prefix': 'Phường', 'name': 'Thịnh Lang'},
     'Hữu Nghị': {'prefix': 'Phường', 'name': 'Hữu Nghị'},
     'Tân Thịnh': {'prefix': 'Phường', 'name': 'Tân Thịnh'},
     'Đồng Tiến': {'prefix': 'Phường', 'name': 'Đồng Tiến'},
     'Phương Lâm': {'prefix': 'Phường', 'name': 'Phương Lâm'},
     'Yên Mông': {'prefix': 'Xã', 'name': 'Yên Mông'},
     'Quỳnh Lâm': {'prefix': 'Phường', 'name': 'Quỳnh Lâm'},
     'Dân Chủ': {'prefix': 'Phường', 'name': 'Dân Chủ'},
     'Hòa Bình': {'prefix': 'Xã', 'name': 'Hòa Bình'},
     'Thống Nhất': {'prefix': 'Phường', 'name': 'Thống Nhất'},
     'Kỳ Sơn': {'prefix': 'Phường', 'name': 'Kỳ Sơn'},
     'Thịnh Minh': {'prefix': 'Xã', 'name': 'Thịnh Minh'},
     'Hợp Thành': {'prefix': 'X

In [18]:
def generate_house_address(with_sublevel=True, sublevel=2) -> str:
    
    num_list = [str(random.randrange(1, 200)) for _ in range(sublevel)]
    result = str()
    for i in range(sublevel):
        result += num_list[i]
        if i < sublevel-1:
            result += '/'
    
    return result


def generate_street_name(min_name_word_len=3, add_prefix=True, abbr_prefix=False, abbr_name=False) -> str:
    
    name = str()
    name_word_len = 0
    while name_word_len < min_name_word_len:
        words = random.choice(vn_words).split()
        name_word_len += len(words)
        name += ' ' + ' '.join(word.capitalize() for word in words)
    
    name = get_abbre(name.strip()) if abbr_name else name.strip()
    prefix = get_abbre("Đường") if abbr_prefix else "Đường"
    
    if add_prefix:
        return prefix + ' ' + name
    else:
        return name

    
def generate_hamlet_name(min_name_word_len=2, add_prefix=True, abbr_prefix=False, abbr_name=False) -> str:
    
    prefixes = ["Thôn", "Xóm", "Ấp", "Đội"]
    
    choose_number = random.random() <= 0.5
    if not choose_number:
        name =  generate_street_name(min_name_word_len, add_prefix=False)
    else:
        name = str(random.randrange(1, 30))
    
    name = get_abbre(name) if abbr_name else name
    prefix = random.choice(prefixes)
    prefix = get_abbre(prefix) if abbr_prefix else prefix
    
    if add_prefix | choose_number:
        return prefix + ' ' + name
    else:
        return name

def generate_quarter_name(min_name_word_len=2, add_prefix=True, abbr_prefix=False, abbr_name=False) -> str:
    
    choose_number = random.random() <= 0.1
    if not choose_number:
        name =  generate_street_name(min_name_word_len, add_prefix=False)
    else:
        name = str(random.randrange(1, 30))
    
    name = get_abbre(name) if abbr_name else name
    prefix = "Khu phố"
    prefix = get_abbre(prefix) if abbr_prefix else prefix
    
    if add_prefix | choose_number:
        return prefix + ' ' + name
    else:
        return name
    
    
def generate_address_below_ward_level(add_house_address=True, # so nha
                                      add_street=True,        # ten duong
                                      add_hamlet=True,        # thon, xom, ap, doi
                                      add_quarter=True        # khu pho
                                     ) -> {
    'address': str,
    'street': str,
    'hamlet': str,
    'quarter': str
}:
    add_prefix_street = random.random() <= 0.5
    add_prefix_hamlet = random.random() <= 0.5
    add_prefix_quarter = random.random() <= 0.5
    
    abbre_params_street = {
        'abbr_prefix': random.random() <= 0.3, 
        'abbr_name': random.random() <= 0.1
    }
    abbre_params_hamlet = {
        'abbr_prefix': random.random() <= 0.3, 
        'abbr_name': random.random() <= 0.1
    }
    abbre_params_quarter = {
        'abbr_prefix': random.random() <= 0.3, 
        'abbr_name': random.random() <= 0.1
    }
    
    return {
        'address': generate_house_address() 
            if add_house_address else None,
        'street': generate_street_name(add_prefix=add_prefix_street, **abbre_params_street) 
            if add_street else None,
        'hamlet': generate_hamlet_name(add_prefix=add_prefix_hamlet, **abbre_params_hamlet) 
            if add_hamlet else None,
        'quarter': generate_quarter_name(add_prefix=add_prefix_quarter, **abbre_params_quarter) 
            if add_quarter else None
    }
    
    
def generate_address_from_ward_level_onward(ward, district, province,
                                            add_ward=True, add_dist=True, add_prov=True) -> {
    'ward': str,
    'district': str,
    'province': str
}:
    result = {    
        'ward': None,
        'district': None,
        'province': None
    }
    
    ward_name = ward['name']
    ward_prefix = ward['prefix']
    dist_name = district['name']
    dist_prefix = district['prefix']
    prov_name = province['name']
    prov_prefix = province['prefix']
    
    abbr_prefix = random.random() < 0.3
    abbr_name = random.random() < 0.1
    ward_name = get_abbre(ward_name) if abbr_name else ward_name
    ward_prefix = get_abbre(ward_prefix) if abbr_prefix else ward_prefix
    
    abbr_prefix = random.random() < 0.3
    abbr_name = random.random() < 0.1
    dist_name = get_abbre(dist_name) if abbr_name else dist_name
    dist_prefix = get_abbre(dist_prefix) if abbr_prefix else dist_prefix
    
    abbr_prefix = random.random() < 0.3
    abbr_name = random.random() < 0.1
    prov_name = get_abbre(prov_name) if abbr_name else prov_name
    prov_prefix = get_abbre(prov_prefix) if abbr_prefix else prov_prefix
    
    if add_ward:
        result['ward'] = (ward_prefix + ' ' if random.random() <= 0.5 else '') + ward_name
    if add_dist:
        result['district'] = (dist_prefix + ' ' if random.random() <= 0.5 else '') + dist_name
    if add_prov:
        result['province'] = (prov_prefix + ' ' if random.random() <= 0.5 else '') + prov_name
    
    return result
        

In [19]:
print(generate_house_address())
print(generate_street_name(min_name_word_len=3, abbr_prefix=random.random() < 0.5, abbr_name=random.random() < 0.5))
print(generate_hamlet_name(min_name_word_len=2, abbr_prefix=random.random() < 0.5, abbr_name=random.random() < 0.5))
print(generate_quarter_name(min_name_word_len=2, abbr_prefix=random.random() < 0.5, abbr_name=random.random() < 0.5))

141/99
Đường Liễn Tiếp Viện
Xóm 16
K. Phố K. Sai


In [20]:
generate_address_below_ward_level()

{'address': '42/91',
 'street': 'Đường Chì Dâu Gia',
 'hamlet': 'Ấp Truyện Tố Giác',
 'quarter': 'Lạng Bợm'}

In [21]:
selection = get_prov_dist_ward_level_region(region_tree)
generate_address_from_ward_level_onward(**selection)

{'ward': "Ea H'đinh", 'district': "Huyện Cư M'gar", 'province': 'Đắk Lắk'}

In [22]:
def generate_fake_address(add_house_address=True, # so nha
                          add_street=True,        # ten duong
                          add_hamlet=True,        # thon, xom, ap, doi
                          add_quarter=True,       # khu pho
                          add_ward=True, add_dist=True, add_prov=True) -> (str, {}):
    
    below = generate_address_below_ward_level(
        add_house_address=add_house_address,
        add_street=add_street,
        add_hamlet=add_hamlet,
        add_quarter=add_quarter
    )
    
    selection = get_prov_dist_ward_level_region(region_tree)
    above = generate_address_from_ward_level_onward(
        **selection,
        add_ward=add_ward, add_dist=add_dist, add_prov=add_prov
    )
    
    combined = {**below, **above}
    
    part_list = []
    for addr_part in combined.values():
        if addr_part is not None:
            part_list.append(addr_part)
    
    return (', '.join(part_list), selection)

In [23]:
generate_fake_address()

('127/28, BMTTự, Thôn 4, Hải Hà, Phú Ninh, Tam Nông, Đồng Tháp',
 {'province': {'prefix': 'Tỉnh',
   'name': 'Đồng Tháp',
   'districts': {'Cao Lãnh': {'prefix': 'Thành phố',
     'name': 'Cao Lãnh',
     'wards': {'11': {'prefix': 'Phường', 'name': '11'},
      '1': {'prefix': 'Phường', 'name': '1'},
      '2': {'prefix': 'Phường', 'name': '2'},
      '4': {'prefix': 'Phường', 'name': '4'},
      '3': {'prefix': 'Phường', 'name': '3'},
      '6': {'prefix': 'Phường', 'name': '6'},
      'Mỹ Ngãi': {'prefix': 'Xã', 'name': 'Mỹ Ngãi'},
      'Mỹ Tân': {'prefix': 'Xã', 'name': 'Mỹ Tân'},
      'Mỹ Trà': {'prefix': 'Xã', 'name': 'Mỹ Trà'},
      'Mỹ Phú': {'prefix': 'Phường', 'name': 'Mỹ Phú'},
      'Tân Thuận Tây': {'prefix': 'Xã', 'name': 'Tân Thuận Tây'},
      'Hoà Thuận': {'prefix': 'Phường', 'name': 'Hoà Thuận'},
      'Hòa An': {'prefix': 'Xã', 'name': 'Hòa An'},
      'Tân Thuận Đông': {'prefix': 'Xã', 'name': 'Tân Thuận Đông'},
      'Tịnh Thới': {'prefix': 'Xã', 'name': 'Tịnh T

In [24]:
for i in range(10):
    param = {
        'add_house_address': random.random() <= 0.6,
        'add_street': random.random() <= 0.8,
        'add_hamlet': random.random() <= 0.5,
        'add_quarter': random.random() <= 0.5,
        'add_ward': random.random() <= 0.9,
        'add_dist': random.random() <= 0.9,
        'add_prov': random.random() <= 0.9,
    }
    addr, ground_truth = generate_fake_address(**param)
    addr = deteriorate_string(addr, 2)
    #addr = expand_space_within_string(addr, (1, 4), include_newline=True)
    print(i, ')--------------------')
    print(addr)
    print(get_prov_dist_ward_level_region_name(ground_truth))

0 )--------------------
Thiên Thần Cứng Cỏi, Khu phố Miễn Arừ, Phường Lộc Hưng, Tr\ng Bàng, Tây Ninh
{'province': 'Tây Ninh', 'district': 'Trảng Bàng', 'ward': 'Lộc Hưng'}
1 )--------------------
67/34, Đường Quẻ 7hôm Sáp Nhập, Khu(phố Rày Trung, Huyện Đức Huệ
{'province': 'Long An', 'district': 'Đức Huệ', 'ward': 'Mỹ Thạnh Bắc'}
2 )--------------------
Đường CQCN, Bưu Phí, ĐônD Sơn, Huyện Đô Lương, Ngh% An
{'province': 'Nghệ An', 'district': 'Đô Lương', 'ward': 'Đông Sơn'}
3 )--------------------
Đường P.Â.T.T., Xóm 28, Khu phố Giờ Phút, Sôngz Đà,  Thị Xã Mường Lay, Điện riên
{'province': 'Điện Biên', 'district': 'Thị Xã Mường Lay', 'ward': 'Sông Đà'}
4 )--------------------
11~/47 ,Xóm 25, Hiệp Hòa, Huyện Đức Hòa, Tỉnh Long An
{'province': 'Long An', 'district': 'Đức Hòa', 'ward': 'Hiệp Hòa'}
5 )--------------------
Ấp 26, Phahn Thây, Búng Lao, ĐiệnqBiên
{'province': 'Điện Biên', 'district': 'Mường Ảng', 'ward': 'Búng Lao'}
6 )--------------------
145/145, Xúc Ph2m, Đắk Som, Tỉ6nh Đắ